# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [35]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [7]:
cities = pd.read_csv("../output_data/Output_CityWeatherSearch.csv", index_col=0)

cities.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,los llanos de aridane,28.6585,-17.9182,62.60,72,40,5.75,ES,1611087484
1,longhua,23.6137,114.1919,53.51,51,60,2.08,CN,1611087484
2,ushuaia,-54.8000,-68.3000,62.60,45,20,4.61,AR,1611087217
3,trapani,37.8333,12.6667,55.00,93,20,6.91,IT,1611087484
4,rikitea,-23.1203,-134.9692,77.94,78,49,16.71,PF,1611087484


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [13]:
# configure gmaps
gmaps.configure(api_key=g_key)

# Store 'Lat' and 'Lng' into  locations 
locations = cities[["Lat", "Lng"]].astype(float)
wgt = cities["Humidity"].astype(int)

In [16]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=wgt, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 3)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

#### My Weather Criteria

- A max temperature lower than 75 degrees but higher than 30.
- Wind speed less than 20 mph.
- Low to moderate cloudiness.
- Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.


In [39]:
criteria = cities.loc[(cities["Max Temp"] <= 75.0) & (cities["Max Temp"] > 65.0),]

criteria = criteria.loc[criteria["Wind Speed"] < 20,]
criteria = criteria.loc[criteria["Cloudiness"] < 20,]

criteria.count()

City          25
Lat           25
Lng           25
Max Temp      25
Humidity      25
Cloudiness    25
Wind Speed    25
Country       25
Date          25
dtype: int64

In [40]:
criteria.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
37,mulanur,10.7667,77.7167,70.36,80,0,4.47,IN,1611087490
47,mar del plata,-38.0023,-57.5575,73.40,64,0,10.36,AR,1611087552
57,marsh harbour,26.5412,-77.0636,69.40,57,0,12.06,BS,1611087554
61,ruwi,23.5996,58.5507,69.80,40,0,3.44,OM,1611087555
65,salalah,17.0151,54.0924,71.60,40,0,4.61,OM,1611087556


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [90]:
hotel_df = criteria.reset_index(drop=True)
hotel_df["Hotel Name"] = ""

url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# set parameters for hotel search
params = { 
    "rankby": "distance",
    "type": "lodging",
    "key": g_key,
}



for n in range(len(hotel_df)):
    lat = hotel_df.iloc[n,1]
    long = hotel_df.iloc[n,2]
    
    params["location"] = str(lat)+","+str(long)
    
    response = requests.get(url, params=params).json()
    
    if response['status'] == "ZERO_RESULTS":
        hotel_df.loc[n,'Hotel Name'] = "No Hotels"
    else:
        hotel_df.loc[n,'Hotel Name'] = response['results'][0]['name']


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,mulanur,10.7667,77.7167,70.36,80,0,4.47,IN,1611087490,Shri Bhagavathi Amman Catering Services
1,mar del plata,-38.0023,-57.5575,73.40,64,0,10.36,AR,1611087552,Balneario Parador Las Toscas
2,marsh harbour,26.5412,-77.0636,69.40,57,0,12.06,BS,1611087554,Ambassador Inn
3,ruwi,23.5996,58.5507,69.80,40,0,3.44,OM,1611087555,Al Wadi Al Kabir Industrial Estate
4,salalah,17.0151,54.0924,71.60,40,0,4.61,OM,1611087556,Muscat International Hotel plaza Salalah


In [94]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [96]:
# Add marker layer ontop of heat map
wgt = hotel_df["Humidity"].astype(int)

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=wgt, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 3)

fig.add_layer(heat_layer)

markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))